In [1]:
# Import libraries
import pandas as pd
import sys
import os

In [2]:
# Get the current working directory
current_dir = os.getcwd()
# Move up one level from the current directory
parent_dir = os.path.dirname(current_dir)
# Change directory into data directory
data_dir = os.path.join(parent_dir, 'data')

In [3]:
sys.path.append(parent_dir)

# 1. Extract

In [4]:
df_checkout_attempts = pd.read_csv(os.path.join(data_dir, 'checkout_attempts.csv'), index_col=None)
df_order_status = pd.read_csv(os.path.join(data_dir, 'order_status.csv'), index_col=None)

In [5]:
df_checkout_attempts.head()

,customer_id,checkout_started_at,checkout_completed_at,order_id,customer_added_non_med_to_order,order_total_dollars
0,1177.0,2023-11-01 06:45:18.000,NaN,NaN,NaN,NaN
1,6119.0,2023-11-01 06:47:35.000,NaN,NaN,NaN,NaN
2,8389.0,2023-11-01 06:50:25.000,2023-11-01 07:05:16.000,10000.0,False,11.75
3,6811.0,2023-11-01 06:55:26.000,2023-11-01 06:59:06.000,10001.0,False,19.83
4,9441.0,2023-11-01 06:59:34.000,2023-11-01 07:06:17.000,10002.0,False,21.59


In [6]:
df_order_status.head()

,order_id,customer_id,delivery_status,delivered_date
0,10000.0,8389.0,delivered,2023-11-04 00:00:00.000
1,10001.0,6811.0,delivered,2023-11-02 00:00:00.000
2,10002.0,9441.0,delivered,2023-11-03 00:00:00.000
3,10003.0,1125.0,delivered,2023-11-03 00:00:00.000
4,10004.0,7838.0,delivered,2023-11-02 00:00:00.000


# Transform

In [7]:
df_checkout_attempts.dtypes

customer_id                        float64
checkout_started_at                 object
checkout_completed_at               object
order_id                           float64
customer_added_non_med_to_order     object
order_total_dollars                float64
dtype: object

In [8]:
df_checkout_attempts['customer_id'] = df_checkout_attempts['customer_id'].astype('Int64')
df_checkout_attempts['checkout_started_at'] = pd.to_datetime(df_checkout_attempts['checkout_started_at'])
df_checkout_attempts['checkout_completed_at'] = pd.to_datetime(df_checkout_attempts['checkout_completed_at'], errors='coerce')
df_checkout_attempts['order_id'] = df_checkout_attempts['order_id'].astype('Int64')
df_checkout_attempts['customer_added_non_med_to_order'] = df_checkout_attempts['customer_added_non_med_to_order'].astype('boolean')
df_checkout_attempts['order_total_dollars'] = pd.to_numeric(df_checkout_attempts['order_total_dollars'], errors='coerce')

In [9]:
df_checkout_attempts.dtypes

customer_id                                 Int64
checkout_started_at                datetime64[ns]
checkout_completed_at              datetime64[ns]
order_id                                    Int64
customer_added_non_med_to_order           boolean
order_total_dollars                       float64
dtype: object

In [10]:
df_order_status.dtypes

order_id           float64
customer_id        float64
delivery_status     object
delivered_date      object
dtype: object

In [11]:
df_order_status['order_id'] = df_order_status['order_id'].astype('Int64')
df_order_status['customer_id'] = df_order_status['customer_id'].astype('Int64')
df_order_status['delivery_status'] = df_order_status['delivery_status'].astype('category')
df_order_status['delivered_date'] = pd.to_datetime(df_order_status['delivered_date'])

# 3.Load

In [12]:
import pyodbc
import db_config as cfg

In [13]:
print(cfg.DATABASE_NAME)

ALTO_PHARMACY


In [14]:
# Test out connection
conn_str = pyodbc.connect(
    'DRIVER={SQL Server};'
    F'SERVER={cfg.SERVER_NAME};'
    F'DATABASE={cfg.DATABASE_NAME};'
    'Trusted_Connection=yes;'
)

In [15]:
def get_column_types(df):
    # Map Pandas data types to SQL Server data types
    type_map = {
        'object': 'NVARCHAR(MAX)',
        'int64': 'BIGINT',
        'float64': 'FLOAT',
        'datetime64[ns]': 'DATETIME2',
        'bool': 'BIT'
    }
    return [type_map.get(str(dt), 'NVARCHAR(MAX)') for dt in df.dtypes]

In [16]:
def load_table(df, table_name, schema_name):
    # Define the connection string
    conn_str = (
        f'DRIVER={{SQL Server}};'
        f'SERVER={cfg.SERVER_NAME};'
        f'DATABASE={cfg.DATABASE_NAME};'
        'Trusted_Connection=yes;'
    )

    # Create a pyodbc connection using the connection string
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Drop the table if it already exists
    cursor.execute(f"IF OBJECT_ID('{schema_name}.{table_name}', 'U') IS NOT NULL DROP TABLE {schema_name}.{table_name}")
    conn.commit()

    # Create the table schema
    columns = ', '.join([f'{col} {dtype}' for col, dtype in zip(df.columns, get_column_types(df))])
    create_table_sql = f"CREATE TABLE {schema_name}.{table_name} ({columns})"
    cursor.execute(create_table_sql)
    conn.commit()

    # Insert the data into the table
    insert_sql = f"INSERT INTO {schema_name}.{table_name} VALUES ({','.join(['?'] * len(df.columns))})"
    for _, row in df.iterrows():
        # Convert NaT in datetime columns to None to keep null values in SQL Server
        row = [None if pd.isnull(x) else x for x in row.tolist()]
        cursor.execute(insert_sql, *row)
    conn.commit()

    cursor.close()
    conn.close()
    print(f"DataFrame loaded successfully into {schema_name}.{table_name}")

In [17]:
df_order_status

,order_id,customer_id,delivery_status,delivered_date
0,10000,8389,delivered,2023-11-04
1,10001,6811,delivered,2023-11-02
2,10002,9441,delivered,2023-11-03
3,10003,1125,delivered,2023-11-03
4,10004,7838,delivered,2023-11-02
...,...,...,...,...
7151,17151,8227,pending,NaT
7152,17152,4970,delivered,2024-01-22
7153,17153,8579,delivered,2024-01-22
7154,17154,6265,delivered,2024-01-22


In [18]:
load_table(df_order_status, 'order_status', 'modealto')

DataFrame loaded successfully into modealto.order_status


In [19]:
load_table(df_checkout_attempts, 'checkout_attempts', 'modealto')

DataFrame loaded successfully into modealto.checkout_attempts
